In [1]:
import requests
from bs4 import BeautifulSoup as bs
import os
requests.packages.urllib3.disable_warnings() 


In [12]:
def parse_cookies(c_string, sep=";"):
    key_values = c_string.split(sep)
    return {key_value.split('=')[0]: key_value.split('=')[1] for key_value in key_values}

In [102]:
def get_all_results(cookies, step=1000):
    url = "https://nouveau-europresse-com.accesdistant.sorbonne-universite.fr/DocumentList/GetNextPage"

    params = {
        'listType': 1,
        'pageNo': 0,
        'docPerPage': step
    }
    res = requests.get('https://nouveau-europresse-com.accesdistant.sorbonne-universite.fr/DocumentList/GetNextPage', 
                       cookies=cookies, 
                       params=params, verify=False)
    return res.content.decode()

In [60]:
def get_validation_token(c_string):
    url = "https://nouveau-europresse-com.accesdistant.sorbonne-universite.fr/Search/AdvancedMobile"
    res = requests.get(url, verify=False, cookies=parse_cookies(c_string))
    return res.content.decode().split('hidden" value="')[1].split('" />')[0]

In [68]:
form = {'__RequestVerificationToken': 'wBDlazGXTAj1jqy9F60YkGQ9ygnm7S1YjLf4eBkKh7R6OerQMU4pq5so05qW_5tsJtqyVvr0zLt4WgMh0',
 'Keywords': 'TEXT%3D+Meat+%26+Livestock',
 'CriteriaKeys%5B0%5D.Operator': '%26',
 'CriteriaKeys%5B0%5D.Key': 'TIT_HEAD',
 'CriteriaKeys%5B0%5D.Text': '',
 'CriteriaKeys%5B1%5D.Operator': '%26',
 'CriteriaKeys%5B1%5D.Key': 'LEAD',
 'CriteriaKeys%5B1%5D.Text': '',
 'CriteriaKeys%5B2%5D.Operator': '%26',
 'CriteriaKeys%5B2%5D.Key': 'AUT_BY',
 'CriteriaKeys%5B2%5D.Text': '',
 'sources': '0',
 'CriteriaSet': '198495',
 'sourcesFilter': '',
 'PostedFilters.FiltersIDs': '8002',
 'DateFilter.DateRange': '10',
 'DateFilter.DateStart': '2012-5-23',
 'DateFilter.DateStop': '2022-03-23',
 'SourcesForm': '0'}
url = "https://nouveau-europresse-com.accesdistant.sorbonne-universite.fr/Search/AdvancedMobile"
# url = "http://localhost:8000"
res = requests.post(url, data=form, cookies=parse_cookies(cookie_string), verify=False)
print(res.status_code)
res.content

200


b'<!DOCTYPE html>\r\n<html class="no-js">\r\n<head>\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n\t<title>Europresse</title>\r\n\t<meta name="description" content="Diffusion d&#39;information de presse sur Internet" />\r\n\t<meta name="keywords" content="Cision, articles, archives, journaux, moteur, recherche, revue, presse" />\r\n\t<meta name="viewport" content="width=device-width, initial-scale=1" />\r\n\r\n\t<link href="/bundles/css?v=M1oJwUmwT7scpFflRL3WEk2nizlbFllMjWeyGAJL54Y1" rel="stylesheet"/>\r\n<link href="/bundles/css/jquery-ui?v=q99EwJe4nVNrtf0dn1Ij_yo4_qnk-09bpJ3tw9qtLu01" rel="stylesheet"/>\r\n\r\n\t<link href="https://fonts.googleapis.com/css?family=Montserrat:300,400,500,600" rel="stylesheet">\r\n\t\r\n\t<link href="/bundles/css/result/mobile?v=kVxxlVGiA2ndSYsJPuwLdVdpd8_nqSQnc34sp8eygig1" rel="stylesheet"/>\r\n<link href="/bundles/css/document?v=Fb-PC6CsDFryoN1wTA2DHZ2r7XDqc4epUC7ShquOvnA1" rel="stylesheet"/>\r\n<link href="/bundles/css/stats?v=1zIKM

In [50]:
def parse_body(body_str):
    key_values = body_str.split("&")
    return {key_value.split('=')[0]: key_value.split('=')[1] for key_value in key_values}

### Try with selenium instead

First perform a request manually on the website, set a start and en date and sort by older first
Then get the 1000 elements
Then we'll try to change the date automatically and start again

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

In [4]:
# create a new firefox driver
profile = webdriver.FirefoxProfile()
driver = webdriver.Firefox(profile)

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_5760\483745394.py:2: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  profile = webdriver.FirefoxProfile()
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_5760\483745394.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  driver = webdriver.Firefox(profile)


In [5]:
driver.get('https://accesdistant.sorbonne-universite.fr/login?url=https://nouveau.europresse.com/access/ip/default.aspx?un=UPMCT_1')

In [115]:
# on fait les manips de connexion et la première recherche à la main
# puis on récupère les cookies  pour stocker les  résultats
all_cookies = driver.get_cookies()
cookies = {}
for cookie in all_cookies:
    cookies[cookie['name']] = cookie['value']
cookies

{'SUAGIMUS': 'TRACEAGIMUS-12223-7AM-CBdCrufqage-M8a6VGCiRXYSVKj5inDx-fx3-auth3.ent.upmc.fr',
 'ezproxy': 'nYpaksoV2XltQOV',
 'mp_116bb615db02bfd2800b004362b744fe_mixpanel': '%7B%22distinct_id%22%3A%20%2254510%22%2C%22%24device_id%22%3A%20%2217fb7e184f2216-0552f6d6de35cf-4c3e2d72-384000-17fb7e184f3c7a%22%2C%22%24initial_referrer%22%3A%20%22https%3A%2F%2Fauth.sorbonne-universite.fr%2F%22%2C%22%24initial_referring_domain%22%3A%20%22auth.sorbonne-universite.fr%22%2C%22__mps%22%3A%20%7B%7D%2C%22__mpso%22%3A%20%7B%7D%2C%22__mpus%22%3A%20%7B%7D%2C%22__mpa%22%3A%20%7B%7D%2C%22__mpu%22%3A%20%7B%7D%2C%22__mpr%22%3A%20%5B%5D%2C%22__mpap%22%3A%20%5B%5D%2C%22%24user_id%22%3A%20%2254510%22%7D'}

In [134]:
# on récupère les résultats
soup = bs(get_all_results(cookies))                #make BeautifulSoup
prettyHTML = soup.prettify()
prettyHTML
mydivs = soup.find_all("span", {"class": "details"})

In [159]:
import re
reg = re.compile('checkbox-container selection-visible.*')
all_el = soup.find_all("div", {"class" : reg})
len(all_el)

1000

In [160]:
all_names = [el.get('name') for el in all_el]

In [161]:
len(all_names)

1000

In [135]:
year, month, day = mydivs[-1].contents[0].strip().split('-')
# this gets rid of trailing 0
month = str(int(month))
day = str(int(month))
print(year, month, day)

2021 8 8


In [124]:
# ensuite on modifie la recherche
driver.get('https://nouveau-europresse-com.accesdistant.sorbonne-universite.fr/Search/AdvancedMobile')

In [130]:
select_day = Select(driver.find_element(By.XPATH, '/html/body/section/form/div/div[2]/div[1]/div[5]/div[3]/div/div[2]/span[1]/span/select[1]'))
select_month = Select(driver.find_element(By.XPATH, '/html/body/section/form/div/div[2]/div[1]/div[5]/div[3]/div/div[2]/span[1]/span/select[2]'))
select_year = Select(driver.find_element(By.XPATH, '/html/body/section/form/div/div[2]/div[1]/div[5]/div[3]/div/div[2]/span[1]/span/select[3]'))

In [131]:
# change values
select_day.select_by_value(day)
select_month.select_by_value(month)
select_year.select_by_value(year)

In [132]:
# submit new form
btn = driver.find_element_by_id('btnSearch')
btn.click()


C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_6312\3584794680.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  btn = driver.find_element_by_id('btnSearch')


In [103]:
from bs4 import BeautifulSoup as bs


In [101]:
import re
reg = re.compile('checkbox-container selection-visible.*')
all_el = soup.find_all("div", {"class" : reg})
len(all_el)
all_names = [el.get('name') for el in all_el]

b'   <div class="pageSeparator"></div>        <div class="docListItem" data-item-id="0" id="doc0" themeId="0">   <div id="chbdoc-0-0" class="checkbox-container selection-visible" name="news&#183;20120423&#183;FBUR&#183;005" index="0"></div>  <div class="docContent"> <div class="docList-header"> <span class="icon-logo-container"><img src="/images/interface/result/ico_pressecrite_small.png" vspace="10" /></span>&nbsp;<span class="source-type pressecrite">Journaux</span> <input id="doc-name" type="hidden" value="news&#183;20120423&#183;FBUR&#183;005" />     <span class="source-name">Business Recorder</span>   </div> <div class="content-wrapper"> <div class="docLink docList-body">  <span class="Z3988" style="display:none" title="ctx_ver=Z39.88-2004&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=Halal%20meat%20exporters%20oppose%20live%20animals%27%20export&amp;rft.jtitle=Business%20Recorder&amp;rft.title=Business%20Recorder&amp;rft.description=E

In [11]:
soup = bs(res.content)                #make BeautifulSoup
prettyHTML = soup.prettify()   #prettify the html

In [16]:
with open(os.path.join('output', 'page1.html'), 'w', encoding='utf-8') as f:
    f.write(prettyHTML)

In [37]:
url_raw = "https://nouveau-europresse-com.accesdistant.sorbonne-universite.fr/DocumentList/GetNextPage?listType=1&pageNo=0&docPerPage=50"
res = requests.get(url_raw, cookies=parse_cookies(cookie_string2), verify=False)

c:\users\utilisateur\miniconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nouveau-europresse-com.accesdistant.sorbonne-universite.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [178]:
%history

import requests
from bs4 import BeautifulSoup as bs
import os
cookie_string = "SUAGIMUS=TRACEAGIMUS-34029-JCNOH6BP3OTI22iN8MoisXhh3eS3TQPjZRsPunwf-auth3.ent.upmc.fr; ezproxy=kA4PX8M0bYO4Pdx"
def parse_cookies(c_string):
    key_values = c_string.split(';')
    return {key_value.split('=')[0]: key_value.split('=')[1] for key_value in key_values}
url = "https://nouveau-europresse-com.accesdistant.sorbonne-universite.fr/DocumentList/GetNextPage"

params = {
    'listType': 1,
    'pageNo': 0,
    'docPerPage': 1000
}
res = requests.get('https://nouveau-europresse-com.accesdistant.sorbonne-universite.fr/DocumentList/GetNextPage', 
                   cookies=parse_cookies(cookie_string_meat), 
                   params=params, verify=False)
url = "https://nouveau-europresse-com.accesdistant.sorbonne-universite.fr/DocumentList/GetNextPage"

params = {
    'listType': 1,
    'pageNo': 0,
    'docPerPage': 1000
}
res = requests.get('https://nouveau-europresse-com.accesdistant.sorbonne-universi

### Full Script

In [7]:
import os
from tqdm import tqdm
import requests
import time
import re
requests.packages.urllib3.disable_warnings() 
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

In [10]:
def get_all_results(cookies, step=1000):
    url = "https://nouveau-europresse-com.accesdistant.sorbonne-universite.fr/DocumentList/GetNextPage"

    params = {
        'listType': 1,
        'pageNo': 0,
        'docPerPage': step
    }
    res = requests.get('https://nouveau-europresse-com.accesdistant.sorbonne-universite.fr/DocumentList/GetNextPage', 
                       cookies=cookies, 
                       params=params, verify=False)
    return res.content.decode()

In [8]:
reg = re.compile('checkbox-container selection-visible.*')

#profile = webdriver.FirefoxProfile()
#driver = webdriver.Firefox(profile)
#driver.get('https://accesdistant.sorbonne-universite.fr/login?url=https://nouveau.europresse.com/access/ip/default.aspx?un=UPMCT_1')
#input()
total_count = int(driver.find_element_by_class_name('resultOperations-count').text.replace(' ', ''))
current_count = 0
all_parsed_documents = set()

all_cookies = driver.get_cookies()
cookies = {}
for cookie in all_cookies:
    cookies[cookie['name']] = cookie['value']

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_5760\2188633158.py:7: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  total_count = int(driver.find_element_by_class_name('resultOperations-count').text.replace(' ', ''))


In [11]:
with tqdm(total=total_count, position=0, leave=True) as pbar:
    while current_count < total_count:
        soup = bs(get_all_results(cookies))
        prettyHTML = soup.prettify()
        all_dates = [el.contents[0].strip() for el in soup.find_all("span", {"class": "details"})]
        all_documents = [el.get('name') for el in soup.find_all("div", {"class" : reg})]
        file_name = f'{all_dates[0]}_{all_dates[-1]}.html'
        print(f"Downloading f{file_name}")
        with open(os.path.join('output', file_name), 'w', encoding='utf-8') as f:
            f.write(prettyHTML)
        nb_new_documents = len(set(all_documents) - all_parsed_documents)
        current_count += nb_new_documents
        pbar.update(nb_new_documents)
        all_parsed_documents.update(all_documents)

        year, month, day = all_dates[-1].split('-')
        # this gets rid of trailing 0
        month = str(int(month))
        day = str(int(day))

        driver.get('https://nouveau-europresse-com.accesdistant.sorbonne-universite.fr/Search/AdvancedMobile')

        select_day = Select(driver.find_element(By.XPATH, '/html/body/section/form/div/div[2]/div[1]/div[5]/div[3]/div/div[2]/span[1]/span/select[1]'))
        select_month = Select(driver.find_element(By.XPATH, '/html/body/section/form/div/div[2]/div[1]/div[5]/div[3]/div/div[2]/span[1]/span/select[2]'))
        select_year = Select(driver.find_element(By.XPATH, '/html/body/section/form/div/div[2]/div[1]/div[5]/div[3]/div/div[2]/span[1]/span/select[3]'))

        print(f'Setting new values to search: day: {day}, month: {month}, year: {year}')
        select_day.select_by_value(day)
        select_month.select_by_value(month)
        select_year.select_by_value(year)

        btn = driver.find_element(By.ID, 'btnSearch')
        btn.click()

        time.sleep(1)

  4%|██▊                                                                          | 1000/27328 [00:16<07:19, 59.93it/s]

Setting new values to search: day: 18, month: 5, year: 2021


  7%|█████▌                                                                       | 1986/27328 [00:43<09:32, 44.29it/s]

Setting new values to search: day: 23, month: 7, year: 2021


 11%|████████▍                                                                    | 2981/27328 [01:07<09:33, 42.48it/s]

...

Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022


 92%|██████████████████████████████████████████████████████████████████████      | 25181/27328 [51:15<05:56,  6.03it/s]

Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022
Setting new values to search: day: 7, month: 4, year: 2022


 92%|██████████████████████████████████████████████████████████████████████      | 25181/27328 [53:22<04:33,  7.86it/s]


KeyboardInterrupt: 

In [11]:
soup = bs(get_all_results(cookies))
prettyHTML = soup.prettify()
all_dates = [el.contents[0].strip() for el in soup.find_all("span", {"class": "details"})]
all_documents = [el.get('name') for el in soup.find_all("div", {"class" : reg})] 
file_name = f'{all_dates[0]}_{all_dates[-1]}.html'
with open(os.path.join('output', file_name), 'w', encoding='utf-8') as f:
    f.write(prettyHTML)
nb_new_documents = len(set(all_documents) - all_parsed_documents)
current_count += nb_new_documents
pbar.update(current_count)
all_parsed_documents.update(all_documents)

year, month, day = all_dates[-1].split('-')
# this gets rid of trailing 0
month = str(int(month))
day = str(int(month))

In [10]:
print(file_name)
print(all_dates[-1].split('-'))
print(year, month, day)

2022-01-07_2022-02-04.html
['2022', '02', '04']
2022 2 2


In [187]:
driver.get('https://nouveau-europresse-com.accesdistant.sorbonne-universite.fr/Search/AdvancedMobile')

select_day = Select(driver.find_element(By.XPATH, '/html/body/section/form/div/div[2]/div[1]/div[5]/div[3]/div/div[2]/span[1]/span/select[1]'))
select_month = Select(driver.find_element(By.XPATH, '/html/body/section/form/div/div[2]/div[1]/div[5]/div[3]/div/div[2]/span[1]/span/select[2]'))
select_year = Select(driver.find_element(By.XPATH, '/html/body/section/form/div/div[2]/div[1]/div[5]/div[3]/div/div[2]/span[1]/span/select[3]'))

select_day.select_by_value(day)
select_month.select_by_value(month)
select_year.select_by_value(year)

btn = driver.find_element_by_id('btnSearch')
btn.click()

time.sleep(1)

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_6312\2748041495.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  btn = driver.find_element_by_id('btnSearch')
